In [1]:
#Starting the spark session
spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1643812820008_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
#Reading the data from Hdfs
base_df = spark.read.csv('/user/root/atm_data_ingest')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
#Checking its schema
base_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: string (nullable = true)
 |-- _c14: string (nullable = true)
 |-- _c15: string (nullable = true)
 |-- _c16: string (nullable = true)
 |-- _c17: string (nullable = true)
 |-- _c18: string (nullable = true)
 |-- _c19: string (nullable = true)
 |-- _c20: string (nullable = true)
 |-- _c21: string (nullable = true)
 |-- _c22: string (nullable = true)
 |-- _c23: string (nullable = true)
 |-- _c24: string (nullable = true)
 |-- _c25: string (nullable = true)
 |-- _c26: string (nullable = true)
 |-- _c27: string (nullable = tru

In [4]:
#Counting the number of records in data frame
base_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

In [5]:
#Providing schema for the base data frame
from pyspark.sql.types import StructType,StructField,IntegerType,StringType,BooleanType,DoubleType,LongType

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
fileSchema = StructType([ 
StructField('year',IntegerType(),True),
StructField('month',StringType(),True), 
StructField('day', IntegerType(),True),
StructField('weekday', StringType(),True), 
StructField('hour', IntegerType(),True),
StructField('atm_status', StringType(),True),
StructField('atm_id', StringType(),True),
StructField('atm_manufacturer', StringType(),True), #Factor
StructField('atm_location', StringType(),True),
StructField('atm_streetname', StringType(),True),
StructField('atm_street_number' ,IntegerType(),True),
StructField('atm_zipcode' ,IntegerType(),True),
StructField('atm_lat' ,DoubleType(),True),
StructField('atm_lon' ,DoubleType(),True),
StructField('currency',StringType(),True), 
StructField('card_type',StringType(),True),
StructField('transaction_amount',IntegerType(),True),
StructField('service',StringType(),True), 
StructField('message_code',StringType(),True), 
StructField('message_text',StringType(),True), 
StructField('weather_lat' ,DoubleType(),True),
StructField('weather_lon' ,DoubleType(),True),
StructField('weather_city_id',IntegerType(),True), 
StructField('weather_city_name',StringType(),True), 
StructField('temp',DoubleType(),True), 
StructField('pressure',IntegerType(),True), 
StructField('humidity',IntegerType(),True), 
StructField('wind_speed',IntegerType(),True), 
StructField('wind_deg',IntegerType(),True), 
StructField('rain_3h' ,DoubleType(),True),
StructField('clouds_all',IntegerType(),True), 
StructField('weather_id',IntegerType(),True), 
StructField('weather_main',StringType(),True), 
StructField('weather_description',StringType(),True)
])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
#Reading the file with new Schema
base_df =  spark.read.csv('/user/root/atm_data_ingest', header= False, schema = fileSchema )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
#Counting the records in the the df
base_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

In [9]:
#Assigning base df to other df for safer side
Main_df=base_df;

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Creating Dimension tables

In [10]:

#Creating the location dimension table
location_df=Main_df.select("atm_location","atm_streetname","atm_street_number","atm_zipcode","atm_lat","atm_lon")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
#Removing duplicates from location dimension
location_df=location_df.dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
#Verifying count in location dim
location_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

109

In [13]:
from pyspark.sql.types import *
from pyspark.sql.window import Window
import pyspark.sql.functions as F
from pyspark.sql.functions import row_number

#Adding location_id with row number
location_df = location_df.select("atm_location","atm_streetname","atm_street_number","atm_zipcode","atm_lat","atm_lon",F.row_number().over(Window.partitionBy().orderBy(location_df['atm_street_number'])).alias("atm_location_id"))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
location_df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+--------------------+-----------------+-----------+-------+-------+---------------+
|    atm_location|      atm_streetname|atm_street_number|atm_zipcode|atm_lat|atm_lon|atm_location_id|
+----------------+--------------------+-----------------+-----------+-------+-------+---------------+
|       Nordkraft|     Kjellerups Torv|                1|       9000| 57.047|  9.932|              1|
|       Svendborg|  Sankt Nicolai Gade|                1|       5700| 55.058| 10.609|              2|
|NykÃƒÂ¸bing Mors|         Kirketorvet|                1|       7900| 56.795|   8.86|              3|
|     SÃƒÂ¦by Syd|Trafikcenter SÃƒÂ...|                1|       9300| 57.313|  10.45|              4|
|         KÃƒÂ¸ge|      SÃƒÂ¸ndre Alle|                1|       4600| 55.454| 12.181|              5|
+----------------+--------------------+-----------------+-----------+-------+-------+---------------+
only showing top 5 rows

In [15]:
#Reordering columns
location_df = location_df.select("atm_location_id","atm_location","atm_streetname","atm_street_number","atm_zipcode","atm_lat","atm_lon")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
#Increasing number of partitions for parallelism 
location_df = location_df.repartition(4)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
#Writing df to s3 bucket
location_df.write.format('csv').option('header','false').save('s3://etlspark001/location/',mode='overwrite')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Date Dim

In [18]:
#Doing same for creating date dim

date_df=Main_df.select("year","month","day","weekday","hour")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
date_df=date_df.dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
from pyspark.sql.functions import concat, col, lit,to_timestamp

date_df=date_df.withColumn("full_date_time",concat(col("day"), lit("-"), col("month"),lit("-"),col("year"),lit(""),col("hour")))
date_df = date_df.withColumn("full_date_time",to_timestamp(col("full_date_time"),'dd-MMMM-yyyyH'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
date_df = date_df.select("year","full_date_time","month","day","weekday","hour",F.row_number().over(Window.partitionBy().orderBy(date_df['year'])).alias("date_id"))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
#Giving order to df
date_df = date_df.select("date_id","full_date_time","year","month","day","hour","weekday")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
#Verifying count in date dim

date_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

8685

In [24]:
#converting timestamp to string for smooth ingestion into redshift
date_df = date_df.withColumn("full_date_time",date_df["full_date_time"].cast(StringType()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
date_df = date_df.repartition(4)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
date_df.write.format('csv').option('header','false').save('s3://etlspark001/date/',mode='overwrite')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Cardtype dim

In [27]:
cardtype = Main_df.select("card_type")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
cardtype=cardtype.dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
cardtype = cardtype.select("card_type",F.row_number().over(Window.partitionBy().orderBy(cardtype['card_type'])).alias("card_type_id"))
cardtype = cardtype.select("card_type_id","card_type")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
#Verifying count in cardtype dim

cardtype.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

12

In [31]:
cardtype = cardtype.repartition(4)
cardtype.write.format('csv').option('header','false').save('s3://etlspark001/card_type/',mode='overwrite')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Joins base with dims

Joining base with location to get location id in base df

In [32]:
Main_df_joined = Main_df.join(location_df,(Main_df.atm_location ==  location_df.atm_location) & 
                   (Main_df.atm_streetname ==  location_df.atm_streetname) &
                   (Main_df.atm_street_number ==  location_df.atm_street_number) &
                   (Main_df.atm_zipcode ==  location_df.atm_zipcode) &
                   (Main_df.atm_lat ==  location_df.atm_lat) &
                   (Main_df.atm_lon ==  location_df.atm_lon) ,"left")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Joining base with date get date id in base df

In [33]:
Main_df_joined = Main_df_joined.join(date_df,(Main_df.year ==  date_df.year) & 
                   (Main_df.month ==  date_df.month) &
                   (Main_df.day ==  date_df.day) &
                   (Main_df.weekday ==  date_df.weekday) &
                   (Main_df.hour ==  date_df.hour) ,"left")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Joining base with card

In [34]:
Main_df_joined = Main_df_joined.join(cardtype, (Main_df.card_type ==  cardtype.card_type) ,"left")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## ATM DIMENSION

In [35]:
atm_df=Main_df_joined.select("atm_id","atm_manufacturer","atm_location_id") 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
atm_df=atm_df.dropDuplicates()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
#Verifying count in atm dim
atm_df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

113

In [38]:
atm_df = atm_df.select("atm_id","atm_manufacturer","atm_location_id",F.row_number().over(Window.partitionBy().orderBy(atm_df['atm_id'])).alias("atm_number"))
atm_df = atm_df.select("atm_id","atm_number","atm_manufacturer","atm_location_id")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
atm_df = atm_df.repartition(4)
atm_df.write.format('csv').option('header','false').save('s3://etlspark001/atm/',mode='overwrite')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Fact Table

In [40]:
#Creating fact with the help of MainDF(base df)
fact=Main_df_joined.select("atm_id","atm_location_id","date_id","card_type_id","atm_status","currency",
                "service","transaction_amount","message_code","message_text","rain_3h","clouds_all","weather_id",
                "weather_main","weather_description")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
fact.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+---------------+-------+------------+----------+--------+----------+------------------+------------+------------+-------+----------+----------+------------+-------------------+
|atm_id|atm_location_id|date_id|card_type_id|atm_status|currency|   service|transaction_amount|message_code|message_text|rain_3h|clouds_all|weather_id|weather_main|weather_description|
+------+---------------+-------+------------+----------+--------+----------+------------------+------------+------------+-------+----------+----------+------------+-------------------+
|    45|            102|   2813|           7|    Active|     DKK|Withdrawal|              7655|        null|        null|  1.275|        92|       500|        Rain|         light rain|
|    45|            102|   8576|           7|    Active|     DKK|Withdrawal|              5587|        null|        null|  0.645|        92|       500|        Rain|         light rain|
|    45|            102|   1427|           7|    Active|     DKK|Withdrawal

In [43]:
# Adding Trans ID
fact = fact.select("atm_id","atm_location_id","date_id","card_type_id","atm_status","currency","service","transaction_amount","message_code","message_text","rain_3h",
                   "clouds_all","weather_id","weather_main","weather_description",F.row_number().over(Window.partitionBy().orderBy(fact['atm_location_id'])).alias("trans_id"))
#Reordering columns
fact=fact.select("trans_id","atm_id","atm_location_id","date_id","card_type_id","atm_status","currency","service","transaction_amount","message_code","message_text","rain_3h",
                   "clouds_all","weather_id","weather_main","weather_description")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
#Verifying count in atm dim

fact.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

2468572

In [45]:
#Writing fact dataframe to s3
fact = fact.repartition(4)
fact.write.format('csv').option('header','false').save('s3://etlspark001/fact/',mode='overwrite')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…